In [1]:
import argparse
import importlib

import torch
from safetensors.torch import save_file, load_file

from diffusers.pipelines.stable_diffusion.convert_from_ckpt import download_from_original_stable_diffusion_ckpt

In [2]:
ckpt_path="./models/lora_1_5/magicalPotions_version1.ckpt"

In [3]:
pipe = download_from_original_stable_diffusion_ckpt(
    checkpoint_path=ckpt_path,
    scheduler_type='pndm',
)

#pipe.save_pretrained(args.dump_path, safe_serialization=args.to_safetensors)

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
/home/tejag/projects/howdy_notebook/venv/lib/python3.11/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [5]:
import torch.nn as nn
import torch.nn.functional as F
from diffusers.models.attention_processor import LoRAAttnProcessor2_0
from diffusers.loaders import AttnProcsLayers

def create_unet_lora_layers(unet: nn.Module):
    lora_attn_procs = {}
    for name in pipe.unet.attn_processors.keys():
        cross_attention_dim = None if name.endswith("attn1.processor") else unet.config.cross_attention_dim
        if name.startswith("mid_block"):
            hidden_size = unet.config.block_out_channels[-1]
        elif name.startswith("up_blocks"):
            block_id = int(name[len("up_blocks.")])
            hidden_size = list(reversed(unet.config.block_out_channels))[block_id]
        elif name.startswith("down_blocks"):
            block_id = int(name[len("down_blocks.")])
            hidden_size = unet.config.block_out_channels[block_id]
        lora_attn_processor_class = (
            LoRAAttnProcessor2_0 if hasattr(F, "scaled_dot_product_attention") else LoRAAttnProcessor
        )
        lora_attn_procs[name] = lora_attn_processor_class(
            hidden_size=hidden_size, cross_attention_dim=cross_attention_dim
        )
    
    unet_lora_layers = AttnProcsLayers(lora_attn_procs)

In [6]:
unet_lora_layers = create_unet_lora_layers(pipe.unet)

In [12]:
pipe.save_lora_weights(
    save_directory='./models/lora_1_5/',
    unet_lora_layers=unet_lora_layers,safe_serialization=True,
    weight_name='magicalPotions_version1.safetensors',
)

In [2]:
import safetensors.torch as t

In [14]:
t.load_file(filename='./models/lora_1_5/magicalPotions_version1.safetensors')

SafetensorError: Error while deserializing header: InvalidHeaderDeserialization

In [ ]:
t.load_file('./models/lora_1_5/')